# 裁剪方法一

In [18]:
from osgeo import gdal
import os

os.chdir(r'D:\Projects\VsCode\Python\img_processing_system\qgis_image\naip\m_4211161_se_12_1_20160624') 
fn = 'm_4211161_se_12_1_20160624.tif' 

fn_ds = gdal.Open(fn)
gtiff_driver = gdal.GetDriverByName('GTiff') 
res_ds = gtiff_driver.Create("m_4211161_se_12_1_20160624_subset.tif", 5834, 2000, bands = fn_ds.RasterCount)
res_ds.SetProjection(fn_ds.GetProjection()) 
gt = fn_ds.GetGeoTransform()
offset_x, offset_y = 0, 2528
new_gt = ( # 计算得到m_4211161_se_12_1_20160624_subset.tif左上角的坐标
    gt[0] + offset_x * gt[1],
    gt[1],
    gt[2],
    gt[3] + offset_y * gt[5],
    gt[4],
    gt[5]
)
res_ds.SetGeoTransform(new_gt)
for i in range(1,fn_ds.RasterCount+1): 
    in_band = fn_ds.GetRasterBand(i)
    in_band_arr = in_band.ReadAsArray(offset_x, offset_y, win_xsize=5834, win_ysize=2000)
    res_ds.GetRasterBand(i).WriteArray(in_band_arr)

res_ds.FlushCache() 

for i in range(1, res_ds.RasterCount + 1):
    src_band = fn_ds.GetRasterBand(i)
    min_val, max_val = src_band.ComputeRasterMinMax(False)
    dst_band = res_ds.GetRasterBand(i)
    dst_band.SetStatistics(min_val, max_val, 0, 0)

del res_ds, fn_ds

## 存在的问题：在qgis中打开这两张图片，在颜色上，m_4211161_se_12_1_20160624_subset.tif比m_4211161_se_12_1_20160624.tif更淡

# 裁剪方法二

In [20]:
from osgeo import gdal
import os

os.chdir(r'D:\Projects\VsCode\Python\img_processing_system\qgis_image\naip\m_4211161_se_12_1_20160624') 
src_file = 'm_4211161_se_12_1_20160624.tif'
dst_file = 'm_4211161_se_12_1_20160624_subset.tif'

gdal.Translate(
    dst_file,
    src_file,
    srcWin=[0, 2528, 5834, 2000]  # x_offset, y_offset, width, height
)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000018CF1B9E0A0> >